In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.lung import lung

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights
LUNG_WEIGHTS_PATH = os.path.join(ROOT_DIR,'mask_rcnn_lung.h5')

E:\Anaconda3\envs\tensorflow1.5\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda3\envs\tensorflow1.5\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda3\envs\tensorflow1.5\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda3\envs\tensorflow1.5\lib\site-packages\tensorflow\python\framework\dtypes.

## Inference Config

In [2]:
config = lung.LungConfig()

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

# use CPU and leave the GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
TEST_MODE = "inference"

## Load Validation Dataset

In [3]:
LUNG_DIR = os.path.join(ROOT_DIR, "datasets/lung")

# Load validation dataset
dataset = lung.LungDataset()
dataset.load_lung(LUNG_DIR, "val") # datasets/lung/val 

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 57
Classes: ['BG', 'lung']


## Load Model

In [4]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# Set path to lung weights file
weights_path = LUNG_WEIGHTS_PATH 
# weights_path = model.find_last() # use the last trained weights file

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  D:\学习\大四\毕业设计\My_Mask\code\Lung\mask_rcnn_lung.h5


## Compute Dice Function

In [5]:
# get metrics for one sample
def getDice(seg, gt):    # seg and gt are both boolen array,2D
    
    smooth = 1 # avoid division by zero

    # compute intersection and union for calculating metrics
    
    intersection = np.sum(np.multiply(seg,gt)) # element-wise product
    aa = 2*intersection + smooth
    bb = np.sum(seg) + np.sum(gt) + smooth
    
    Dice = aa / bb
    Rs = np.sum(gt)
    Os = np.sum(seg) - intersection
    Us = Rs - intersection
    OR = (Os + smooth) / (Rs + Os + smooth)
    UR = (Us + smooth) / (Rs + Os + smooth)
    
    return (Dice,OR,UR)

## Compute Dice on Validation Dataset

In [6]:
computation_results = np.zeros((len(dataset.image_ids),3)) # save Dice,OR,UR

for i,image_id in enumerate(dataset.image_ids):
    
    # read image input and its ground truth
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))

    # Run object detection
    results = model.detect([image], verbose=1)
    
    # get segmentation results
    pred_mask = results[0]["masks"]
    
    # sumarize on the channel-multi objects
    pred_mask = np.sum(pred_mask,axis=2)
    gt_mask = np.sum(gt_mask,axis=2)
    
    # compute Dice 
    (Dice,OR,UR) = getDice(pred_mask,gt_mask)
    computation_results[i,:] = (Dice,OR,UR) 

image ID: lung.10548316_0000_i.tif (0) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\10548316_0000_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
image ID: lung.10548316_0001_i.tif (1) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\10548316_0001_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:

image ID: lung.10548316_0158_i.tif (16) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\10548316_0158_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
image ID: lung.10548316_0159_i.tif (17) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\10548316_0159_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        mi

image ID: lung.10548316_0312_i.tif (32) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\10548316_0312_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
image ID: lung.10548316_0319_i.tif (33) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\10548316_0319_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        mi

image ID: lung.BA3RGWI0_0129_i.tif (48) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\BA3RGWI0_0129_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
image ID: lung.BA3RGWI0_0133_i.tif (49) D:\学习\大四\毕业设计\My_Mask\code\Lung\datasets/lung\val\images\BA3RGWI0_0133_i.tif
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max: 65535.00000  uint16
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max: 65431.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        mi

## Get Mean Value on Validation Dataset

In [7]:
mDice, mOR, mUR = np.mean(computation_results,axis=0)
print("The Dice on Valiadation Dataset is %f" % mDice)
print("The OR on Valiadation Dataset is %f" % mOR)
print("The UR on Valiadation Dataset is %f" % mUR)

The Dice on Valiadation Dataset is 0.933988
The OR on Valiadation Dataset is 0.085236
The UR on Valiadation Dataset is 0.092524
